In [ ]:
# Initial imports and loading in our data.
# This is (almost) direct from SurveyMonkey, though
# I scrubbed IP addresses and access dates to anonymize.
import numpy as np
import pandas as pd

df = pd.read_csv('aperture_survey_responses.tsv', sep='\t')

In [ ]:
# SurveyMonkey provides an odd nesting of responses when exporting results.
# We'd like to convert this structure to a pandas MultiIndex data frame.
# First, let's find question indices -- adapted from https://stackoverflow.com/a/49584888
indices = [i for i, c in enumerate(df.columns) if not c.startswith('Unnamed')]
slices = [slice(i, j) for i, j in zip(indices, indices[1:] + [None])]
repeats = [len(range(*slice.indices(len(df.columns)))) for slice in slices]

In [ ]:
# Now let's grab all of the questions and each of the options provided as possible responses.
questions = [c for c in df.columns if not c.startswith('Unnamed')]
options = df.iloc[:1].values[0].tolist()

# We can pair each possible response with its associated question...
matched_questions = []
for index, question in enumerate(questions):
    matched_questions += [question] * repeats[index]

# ... and create a new dataframe named 'responses' that correctly pairs questions and responses.
index = pd.MultiIndex.from_arrays([matched_questions, options],
                                  names=('question', 'options'))
data = df.iloc[2:].values
responses = pd.DataFrame(data=data, columns=index)

In [ ]:
# Optional data cleaning -- remove 'Other' responses for easier plotting.
responses.drop('Other (please specify)',
               axis=1, level=1, inplace=True)

In [ ]:
# First question of interest --
responses['Should OHBM have an official publishing platform?',
          'Response'].value_counts(normalize=True)

In [ ]:
# What are respondents willing to pay to sustain Aperture ?
price_per_sub = pd.to_numeric(responses['Per submission:',
                                        'Open-Ended Response']).mean()
price_per_pub = pd.to_numeric(responses['Per publication:',
                                        'Open-Ended Response']).mean()

print('On average, respondents were willing to pay ${:.0f} per submission to Aperture.'.format(price_per_sub))
print('They also indicated they would pay an additional ${:.0f} to publish in Aperture.'.format(price_per_pub))